In [1]:
import tensorflow as tf
from numpy import sqrt



def neural_layer(X, n_neurons, layer_name, apply_activation=True):
    
    n_inputs = int(X.get_shape()[1])
    with tf.name_scope(layer_name):
        std_dev = 2/sqrt(n_inputs)
        init_w = tf.random.truncated_normal((n_inputs, n_neurons),mean=0, stddev=std_dev)
        ## each column is weights of a neuron
        
        w = tf.Variable(init_w, name="w")
        b = tf.Variable(tf.zeros((n_neurons)))
        
        z = tf.matmul(X,w) + b
        
        if apply_activation:
            return tf.nn.relu(z)
        else:
            return z


def create_dnn(X, layer_neurons):
    last_layer = X
    
    with tf.name_scope("dnn"):
        for i,n_neurons in enumerate(layer_neurons):
            layer_name = "input_layer" if i==0 \
                else ("output_layer" if i==len(layer_neurons)-1 else "hidden_layer"+str(i))
            layer = neural_layer(last_layer, n_neurons, layer_name, 
                                 apply_activation=(i!= len(layer_neurons)-1))
            last_layer = layer
        
    return last_layer
    

def create_loss(y,dnn):
    with tf.name_scope("loss"):
        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=y, logits=dnn)

        loss = tf.reduce_mean(xentropy, name="loss")
    return loss


def get_train_op(loss, learning_rate=0.01):
    with tf.name_scope("train"):
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
        train_op = optimizer.minimize(loss)
    return train_op



/Users/devbhadurkhadka/.pyenv/versions/anaconda3-5.2.0/envs/scikit_practice/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


KeyboardInterrupt: 

In [43]:
from tensorflow.examples.tutorials import mnist
mnist_data = mnist.input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [131]:
import numpy as np


from my_libs.tf_checkpoint import CheckpointSaver
from my_libs.tf_graph_saver import ScalerGraphSaver

n_epochs = 100
batch_size = 50
lr=0.01


def perform_gradient_descent(train_data, n_epochs, batch_size):
    tf.reset_default_graph()
    n_inputs = 28*28
    layer_neurons = [300,200,10]
    n_batchs = int(np.ceil(mnist_data.train.num_examples/batch_size))
    
    X = tf.placeholder(tf.float32, (None, n_inputs) , name="X")
    y = tf.placeholder(tf.int64, (None), name="y")

    cur_epoch = tf.Variable(0,dtype=tf.int32, name="cur_epoch")
    inc_cur_epoch = tf.assign(cur_epoch, cur_epoch+1)

    dnn = create_dnn(X,layer_neurons=layer_neurons)
    loss = create_loss(y, dnn)
    train_op = get_train_op(loss,learning_rate=lr)
    
    with tf.name_scope("accuracy"):
        correct_preds = tf.nn.in_top_k(dnn,y,1)
        accuracy = tf.reduce_mean(tf.cast(correct_preds,tf.float32))
    
    init = tf.global_variables_initializer()
    
    
    
    with tf.Session() as sess:
        chkpnt_saver = CheckpointSaver("tf_dnn_example")
        
        if not chkpnt_saver.restore_checkpoint():
            sess.run(init)
        else:
            display("check point restored")
        
        epoch_start = cur_epoch.eval()
        display("epoch_start",epoch_start)
        
        test_images = mnist_data.test.images
        test_labels = mnist_data.test.labels
        
        with ScalerGraphSaver("tf_dnn_example", loss) as graph_saver:
            for epoch in range(epoch_start,n_epochs):
                for batch in range(n_batchs):
                    X_batch, y_batch = train_data.next_batch(batch_size)

                    feed_dict={X:X_batch,y:y_batch}
                    sess.run(train_op, feed_dict=feed_dict)

                sess.run(inc_cur_epoch)
                graph_saver.log_summary(epoch, feed_dict)
                if epoch%20==0 or epoch==n_epochs-1:
                    print("epoch", epoch)
                    chkpnt_saver.save_checkpoint()
                    batch_acc = accuracy.eval(feed_dict=feed_dict)
                    test_acc = accuracy.eval(feed_dict={X:test_images, y:test_labels})
                    display(batch_acc, test_acc)
                    
    def predict(inp):
        with tf.Session() as sess:
            chkpnt_saver = CheckpointSaver("tf_dnn_example")
            chkpnt_saver.restore_checkpoint()
            prob_dist = dnn.eval(feed_dict={X:inp})
#             return np.argmax(prob_dist, axis=1)
            return prob_dist
    
    return predict
                

In [133]:
n_epochs = 150
predict = perform_gradient_descent(mnist_data.train, n_epochs, batch_size)

'check point restored'

'epoch_start'

100

epoch 100


1.0

...

1.0

0.9796

epoch 149


1.0

0.9792

In [123]:
from sklearn.metrics import accuracy_score
predictions = predict(mnist_data.test.images)
# accuracy_score(predictions, mnist_data.test.labels)

In [124]:
pred = np.argmax(predictions, axis=1)
pred

array([7, 2, 1, ..., 4, 5, 6])

In [125]:
accuracy_score(pred, mnist_data.test.labels)

0.9783